In [1]:
# 데이터 붙이기 => 번호 컬럼만 따로 가져오기 => 기존 데이터에 내용 추가해보기

In [402]:
import numpy as np
import pandas as pd
import requests 
import time
from bs4 import BeautifulSoup 
from glob import glob


# 21.09.20 기준 https://kr.investing.com/stock-screener/ 사이트에서 1~10페이지 분량 작업. 총 54페이분량

# file_names = glob("/Users/ppangppang/Desktop/ssac/finance_data/*.csv")
# file_names

# total = pd.DataFrame()
# for file_name in file_names:
#     temp = pd.read_csv(file_name)
#     total = pd.concat([total, temp])
pd.options.display.max_info_columns =200
pd.options.display.max_columns = 200
pd.options.display.max_info_rows =100
pd.options.display.max_rows = 100

In [403]:
# 사용할 데이터만가져오기, 기존 데이터에 있는 종가는 현재 가격이 아님
total.index = range(len(total))
df = total[['종목','기호','거래량']]
df[99:200]

,종목,기호,거래량
99,Cubes,65560,0.00
100,현대무벡스,319400,0.00
101,랩지노믹스,84650,0.00
102,Novarex,194700,0.00
103,한국쉘석유,2960,2.14K
...,...,...,...
195,바른손,18700,0.00
196,메타랩스,90370,215.11K
197,메타바이오메드,59210,0.00
198,모비스,250060,0.00


In [422]:
# 종목만 따로 떼오기. str로 변환안하면 0을 미포함하는 경우가 생김
temp = df['기호'].values
temp2 = list(map(str, temp))

for num in range(len(temp2)-1):
    if len(temp2[num]) ==5:
        temp2[num] = '0' + temp2[num]
    elif len(temp2[num]) ==4:
        temp2[num] = '00' + temp2[num]
    elif len(temp2[num]) ==3:
        temp2[num] = '000' + temp2[num]
    elif len(temp2[num]) ==2:
        temp2[num] = '0000' + temp2[num]
    elif len(temp2[num]) ==1:
        temp2[num] = '00000' + temp2[num]
temp = temp2

In [423]:
# str형으로 변경됐는지 최종확인
for num in range(len(temp)-1):
    if len(temp[num]) !=6:
        print(num)  # 	미래에셋맵스프런티어브라질 8자리로 되어있는 펀드같음 신경 안써도 되느낮료

2455


In [424]:
# 종목코드 변환한것 df에 넣기
temp=np.array(temp)
temp = np.ravel(temp, order='C')
temp = pd.DataFrame({"기호" : temp})
# temp
df1 = df[['종목']]
df2 = df1.join(temp, how='outer')
df3 = df2.join(df[['거래량']], how='outer')
df_set = df3

In [425]:
code= df_set['기호']
len(code)

2658

In [407]:
# 종목번호를 잘못입력해도 연결되고, html 변환도 오류 안뜬다

# url = f'https://finance.naver.com/item/main.nhn?code=000000'
# response = requests.get(url)
# response
# html = BeautifulSoup(response.text, 'html.parser')

In [408]:
# 오류 뜨기전 사용하던 코드
# price = []

# for num in code:
#     url = f'https://finance.naver.com/item/main.nhn?code={num}'

#     print(f'{num}종목 현재가격 크롤링중 입니다.')
#     seed = np.random.randint(100) # seed 설정
#     np.random.seed(seed)    # 시드 생성
#     a = np.random.randint(5)  # 시드에서 난수 생성
#     time.sleep(a)     # 코드실행 지연시키는 코드
    
#     response = requests.get(url)

#     if(response.status_code == requests.codes.ok):
#         html = BeautifulSoup(response.text, 'html.parser')

#     else:
# #         print(f'{num}종목 크롤링 실패, 값을 다시 확인해주세요(금일종가 0원으로 처리)')
#         print(f"크롤링 실패. 종목번호 : {num} 을 확인해 주세요")
# #         price.append('0')
# #         continue

#     price.append(html.select('p.no_today')[0].text)


In [ ]:
# 현재 최종본 / 0으로 시작하는 코드 제대로 크롤링되는지 확인
price = []

for num in code:
    url = f'https://finance.naver.com/item/main.nhn?code={num}'

    print(f'{num}종목 현재가격 크롤링중 입니다.')
    seed = np.random.randint(100) # seed 설정
    np.random.seed(seed)    # 시드 생성
    a = np.random.randint(5)  # 시드에서 난수 생성
    time.sleep(a)     # 코드실행 지연시키는 코드
    
    response = requests.get(url)

    if(response.status_code == requests.codes.ok):
        html = BeautifulSoup(response.text, 'html.parser')
           
    else:
        print(f"크롤링 실패. 종목번호 : {num} 을 확인해 주세요")

    try:
        price.append(html.select('p.no_today')[0].text)

    except:
        print(f'{num}종목 크롤링 실패, 값을 다시 확인해주세요(현재가격 0원으로 처리)')
        price.append('0')

print("크롤링 완료")

041930종목 현재가격 크롤링중 입니다.
101060종목 현재가격 크롤링중 입니다.
187420종목 현재가격 크롤링중 입니다.
060590종목 현재가격 크롤링중 입니다.
357580종목 현재가격 크롤링중 입니다.
092230종목 현재가격 크롤링중 입니다.
290670종목 현재가격 크롤링중 입니다.
000430종목 현재가격 크롤링중 입니다.
007160종목 현재가격 크롤링중 입니다.


In [391]:
# price 정리

code_price =[]

for num in price:
    temp = []
        # 현재가격 크롤링 되지 않는 데이터를 html.select형태로 변환해서 작업해버리자
    if num == '0':
        num = '\n\n0\n0\n\n'
        temp.append(num.split('\n'))
        temp=np.array(temp)
        code_price.append(temp[0,2:3])

    else:
        temp.append(num.split('\n'))
        temp=np.array(temp)
        code_price.append(temp[0,2:3])

code_price = np.ravel(code_price, order='C')
code_price = pd.DataFrame({"현재가격" : code_price})

In [392]:
code_price

,현재가격
0,"48,250"
1,"174,500"
2,"37,150"
3,"150,800"
4,"65,300"
...,...
545,"44,400"
546,"161,000"
547,"587,000"
548,"190,000"


In [393]:
# 현재가격 업데이트
df_daily = df.join(code_price, how='outer')

In [394]:
# 최종확인
df_daily[100:200]

,종목,기호,거래량,현재가격
100,원익QnC,74600,0.00,"26,300"
101,에스앤에스텍,101490,0.00,"33,100"
102,NICE,34310,34.03K,"18,300"
103,인텔리안테크,189300,0.00,"76,500"
104,디오,39840,0.00,"45,650"
105,삼양사우,145995,422.00,"47,700"
106,아미코젠,92040,0.00,"35,300"
107,코웰패션,33290,0.00,"7,820"
108,바텍,43150,0.00,"44,650"
109,아주캐피탈,33660,0.00,0
